In [13]:
import pandas as pd
from tqdm import tqdm
import numpy as np

In [2]:
import sys
sys.path.append('../')
validation_path = ''
train_set_path = ''
authors_validation = ''
output_path = ''

In [3]:
df_test = pd.read_csv(validation_path)
df_test_author_ids = df_test['author_id'].unique()
df_train = pd.read_csv(train_set_path)
author_ids = df_train[df_train['author_id'].isin(df_test_author_ids)]['author_id'].unique()
article_counts = df_train.groupby(['author_id', 'article_id'])['comment_id'].nunique()
article_counts = article_counts.reset_index()
article_ids = df_train['article_id'].unique()
article_to_position = {article_id: i for i, article_id in enumerate(article_ids)}

In [7]:
def prepare_collab():
    df_test = pd.read_csv(validation_path)
    df_test_author_ids = df_test['author_id'].unique()
    df_train = pd.read_csv(train_set_path)
    author_ids = df_train[df_train['author_id'].isin(df_test_author_ids)]['author_id'].unique()
    article_counts = df_train.groupby(['author_id', 'article_id'])['comment_id'].nunique()
    article_counts = article_counts.reset_index()
    article_ids = df_train['article_id'].unique()
    article_to_position = {article_id: i for i, article_id in enumerate(article_ids)}


    # %%
    def get_user_representation(user_id):
        selection = article_counts[article_counts['author_id'] == user_id]
        user_dict = {}
        for index, row in selection.iterrows():
            article_id = row['article_id']
            count = row['comment_id']
            user_dict[article_id] = count
        return user_dict

    # %%
    user_vectors = {}
    for user in tqdm(author_ids):
        user_vectors[user] = get_user_representation(user)
    return user_vectors, article_to_position, len(article_ids)

In [8]:
user_vector_info_dict, article_to_position, vector_size = prepare_collab()

100%|██████████| 47175/47175 [02:35<00:00, 303.56it/s]


In [9]:
vector_size

13419

In [10]:
df_test = pd.read_csv(validation_path)
df_test.index = df_test['comment_id']
comment_id_to_user_id = df_test['author_id'].to_dict()

In [11]:
def get_user_representation_vector(user_id):
    selection = article_counts[article_counts['author_id'] == user_id]
    user_dict = {}
    for index, row in selection.iterrows():
        article_id = row['article_id']
        count = row['comment_id']
        user_dict[article_id] = count
    return user_dict


def get_user_representation(user_id):
    user_info_dict = user_vector_info_dict[user_id]
    out = np.zeros(vector_size)
    for key, value in user_info_dict.items():
        out[article_to_position[key]] = value
    return out


In [14]:
usable_authors_test = np.load(authors_validation)

In [15]:
user_matrix = np.zeros((len(usable_authors_test), len(get_user_representation(usable_authors_test[0]))))

In [16]:
user_matrix.shape

(14457, 13419)

In [17]:
from gensim.models import KeyedVectors

In [18]:
from tqdm import tqdm_notebook

In [19]:
for index, author in enumerate(tqdm(usable_authors_test)):
    user_matrix[index] = get_user_representation(author)

100%|██████████| 14457/14457 [00:00<00:00, 18828.90it/s]


In [20]:
key_vectors = KeyedVectors(vector_size=len(user_matrix[0]))

In [21]:
authors = [str(author) for author in usable_authors_test]

In [22]:
key_vectors.add(authors, user_matrix)

In [23]:
key_vectors.save_word2vec_format(output_path)